# WatsonX.ai Client with Milvus and LangChain

This is the client Watsonx.ai LLM-driven question-answering application with Milvus and LangChain

## Set up the environment
Before you use the sample code in this notebook, you must perform the following setup tasks:

Create a Watson [Machine Learning (WML)](https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/) Service instance (a free plan is offered and information about how to create the instance can be found [here](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-service-instance.html?context=analytics)).

### Install and import dependecies

In [1]:
from IPython.display import clear_output
!pip install pymilvus
!pip install "langchain==0.0.345" 
!pip install wget 
!pip install sentence-transformers 
!pip install chromadb==0.3.22 
!pip install "ibm-watson-machine-learning>=1.0.335" 
!pip install "pydantic>=1.4.0,<2" 
!pip install bs4
!pip install ipywidgets
!pip install towhee towhee.models 
!pip install gradio "pydantic>=1.4.0,<2" datasets
clear_output()


### Prepare the Data

In [14]:
from datasets import load_dataset
from IPython.display import clear_output
dataset = load_dataset("ruslanmv/ai-medical-chatbot")
clear_output()
train_data = dataset["train"]
#For this demo let us choose the first 1000 dialogues
import pandas as pd
df = pd.DataFrame(train_data[:1000])
#df = df[["Patient", "Doctor"]].rename(columns={"Patient": "question", "Doctor": "answer"})
df = df[["Description", "Doctor"]].rename(columns={"Description": "question", "Doctor": "answer"})
# Add the 'ID' column as the first column
df.insert(0, 'id', df.index)
# Reset the index and drop the previous index column
df = df.reset_index(drop=True)
import re
# Clean the 'question' and 'answer' columns
df['question'] = df['question'].apply(lambda x: re.sub(r'\s+', ' ', x.strip()))
df['answer'] = df['answer'].apply(lambda x: re.sub(r'\s+', ' ', x.strip()))
df['question'] = df['question'].str.replace('^Q.', '', regex=True)
# Assuming your DataFrame is named df
max_length = 500  # Due to our enbeeding model does not allow long strings
df['question'] = df['question'].str.slice(0, max_length)
#To use the dataset to get answers, let's first define the dictionary:
#- `id_answer`: a dictionary of id and corresponding answer
id_answer = df.set_index('id')['answer'].to_dict()


## Foundation Models on watsonx

In [15]:
from dotenv import load_dotenv
import os
load_dotenv()
try:
    API_KEY = os.environ.get("API_KEY")
    project_id =os.environ.get("PROJECT_ID")
except KeyError:
    API_KEY: input("Please enter your WML api key (hit enter): ")
    project_id  = input("Please  project_id (hit enter): ")

credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": API_KEY  
}    


### Defining model
You need to specify model_id that will be used for inferencing:



In [16]:
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes
model_id = ModelTypes.GRANITE_13B_CHAT_V2

## Defining the model parameters
We need to provide a set of model parameters that will influence the result:

In [17]:
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import DecodingMethods
parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 500,
    GenParams.STOP_SEQUENCES: ["<|endoftext|>"]
}

LangChain CustomLLM wrapper for watsonx model
Initialize the WatsonxLLM class from Langchain with defined parameters and ibm/granite-13b-chat-v2

In [18]:
from langchain.llms import WatsonxLLM
watsonx_granite = WatsonxLLM(
    model_id=model_id.value,
    url=credentials.get("url"),
    apikey=credentials.get("apikey"),
    project_id=project_id,
    params=parameters
)

### Setup Remote Server
Here we should define the variable `REMOTE_SERVER` just created [here](https://github.com/ruslanmv/Watsonx-Assistant-with-Milvus-as-Vector-Database/blob/master/README.md)

In [3]:
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.embeddings.base import Embeddings
from langchain.vectorstores.milvus import Milvus
from langchain.embeddings import HuggingFaceEmbeddings  # Not used in this example
import pickle

# Loading the pickle file
with open("id_answer.pkl", "rb") as file:
    id_answer = pickle.load(file)
# (Assuming you have the required SentenceTransformer model loaded)
#embeddings = SentenceTransformerEmbeddings()  # Create SentenceTransformer embeddings instance

COLLECTION_NAME='qa_medical'
from dotenv import load_dotenv
import os
load_dotenv()
host_milvus = os.environ.get("REMOTE_SERVER", '127.0.0.1')
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility
connections.connect(host=host_milvus, port='19530')

from pymilvus import Collection, utility
collection = Collection(COLLECTION_NAME)      
collection.load(replica_number=1)
utility.load_state(COLLECTION_NAME)
utility.loading_progress(COLLECTION_NAME)

from towhee import pipe, ops
import numpy as np
def convert_text_to_embedding(input_text):
    # Define the pipeline for text embedding, ensuring float32 output
    embedding_pipe = (
        pipe.input('text')
        .map('text', 'vec', ops.text_embedding.dpr(model_name="facebook/dpr-ctx_encoder-single-nq-base"))
        .map('vec', 'normalized_vec', lambda x: x / np.linalg.norm(x, axis=0))
        .map('normalized_vec', 'float_vec', lambda x: np.array(x, dtype=np.float32))  # Explicitly convert to float32
        .output('float_vec')
    )
    # Get the embedding for the input text
    embedding_queue = embedding_pipe(input_text)
    embedding = embedding_queue.get()
    return embedding[0]  # Return the first (and only) embedding

query = "I have started to get lots of acne on my face, particularly on my forehead. Please help me"

query_embedding = convert_text_to_embedding(query)  # Get embedding for the query sentence

def search_milvus_collection(collection, query_embedding, top_k=10):
    search_params = {"metric_type": "L2", "params": {"nprobe": 10}}
    results = collection.search(data=[query_embedding], anns_field="embedding", param=search_params, limit=top_k)
    return results
# Search the collection
search_results = search_milvus_collection(collection, query_embedding)

def collect_retrieved(search_results, num_docs=10, sort_key="distance", ascending=True, id_answer=None):
  """
  Collects, sorts, and extracts text for retrieved documents.

  Args:
    search_results: The output of a Milvus search result.
    num_docs: The maximum number of documents to collect (default: 10).
    sort_key: The key to sort documents by (default: "distance").
    ascending: Whether to sort in ascending order (default: True).
    id_answer: A dictionary mapping document IDs to their text (optional).

  Returns:
    A list of dictionaries, each containing "id" and "text" keys for the retrieved documents.
  """
  retrieved_data = [(result.id, result.distance) for result in search_results[0]]
  sorted_data = sorted(retrieved_data, key=lambda x: x[1] if sort_key == "distance" else x[0], reverse=not ascending)

  retrieved_docs = []
  for item in sorted_data[:num_docs]:
    doc_id = item[0]
    doc_text = id_answer.get(doc_id, 'Answer not found')  # Retrieve text from the dictionary
    retrieved_docs.append({"id": doc_id, "text": doc_text})

  return retrieved_docs
# Collect top 2 documents sorted by descending relevance
docs = collect_retrieved(search_results, num_docs=1, id_answer=id_answer)
docs[0]['text']

for doc in docs:
  print(f"Document ID: {doc['id']}")
  print(f"Document Text: {doc['text']}")
# Assuming you have initialized watsonx_granite as the LLM


Document ID: 2
Document Text: Hi there Acne has multifactorial etiology. Only acne soap does not improve if ypu have grade 2 or more grade acne. You need to have oral and topical medications. This before writing medicines i need to confirm your grade of acne. For mild grade topical clindamycin or retenoic acud derivative would suffice whereas for higher grade acne you need oral medicines aluke doxycycline azithromycin or isotretinoin. Acne vulgaris Cleansing face with antiacne face wash


Generate a retrieval-augmented response to a question
Build the RetrievalQA (question answering chain) to automate the RAG task.


In [4]:
#import langchain.chains as lc
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_core.documents import Document
from pymilvus import Collection, utility
from towhee import pipe, ops
import numpy as np
from towhee.datacollection import DataCollection
from typing import List

max_input_length = 500  # Maximum length allowed by the model

def get_data(query):
  query_embedding = convert_text_to_embedding(query)  # Get embedding for the query sentence
  # Search the collection
  search_results = search_milvus_collection(collection, query_embedding)
   # Collect top 2 documents sorted by descending relevance
  docs = collect_retrieved(search_results, num_docs=2, id_answer=id_answer)
  return docs

# Create the combined pipe for question encoding and answer retrieval
combined_pipe = (
    pipe.input('question')
        .map('question', 'vec', lambda x: x[:max_input_length])  # Truncate the question if longer than 512 tokens
        .map('vec', 'vec', ops.text_embedding.dpr(model_name='facebook/dpr-ctx_encoder-single-nq-base'))
        .map('vec', 'vec', lambda x: x / np.linalg.norm(x, axis=0))
        .map('vec', 'res', ops.ann_search.milvus_client(host=host_milvus, port='19530', collection_name=COLLECTION_NAME, limit=1))
        .map('res', 'answer', lambda x: [id_answer[int(i[0])] for i in x])
        .output('question', 'answer')
)
class CustomRetriever(BaseRetriever): 
    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        # Perform the encoding and retrieval for a specific question
        ans = combined_pipe(query)
        ans = DataCollection(ans)
        answer=ans[0]['answer']
        answer_string = ' '.join(answer)
        return [Document(page_content=answer_string)]   

query = "I have started to get lots of acne on my face, particularly on my forehead what can I do"
docs=get_data(query)
retriever = CustomRetriever(documents=docs)
relevant_documents = retriever.get_relevant_documents(query)
print(relevant_documents)    



[Document(page_content='Hi there Acne has multifactorial etiology. Only acne soap does not improve if ypu have grade 2 or more grade acne. You need to have oral and topical medications. This before writing medicines i need to confirm your grade of acne. For mild grade topical clindamycin or retenoic acud derivative would suffice whereas for higher grade acne you need oral medicines aluke doxycycline azithromycin or isotretinoin. Acne vulgaris Cleansing face with antiacne face wash')]


# Clean New Retriever

In [39]:
class CustomRetriever:
    def get_relevant_documents(
        self, query: str, 
    ) -> List[Document]:
        # Perform the encoding and retrieval for a specific question
        ans = combined_pipe(query)
        ans = DataCollection(ans)
        answer = ans[0]['answer']
        answer_string = ' '.join(answer)
        return [Document(page_content=answer_string)]
query = "I have started to get lots of acne on my face, particularly on my forehead. What can I do?"
retriever = CustomRetriever()
relevant_documents = retriever.get_relevant_documents(query)
print(relevant_documents)

[Document(page_content='Hi there Acne has multifactorial etiology. Only acne soap does not improve if ypu have grade 2 or more grade acne. You need to have oral and topical medications. This before writing medicines i need to confirm your grade of acne. For mild grade topical clindamycin or retenoic acud derivative would suffice whereas for higher grade acne you need oral medicines aluke doxycycline azithromycin or isotretinoin. Acne vulgaris Cleansing face with antiacne face wash')]


In [40]:
# This will only documents related with the query
query = " I have headacke what can I do"
docs_search =retriever.get_relevant_documents(query)
docs_search

[Document(page_content='Hello. I understand your concern and would first recommend performing an EEG (electroencephalography) and a brain MRI to exclude any possible brain lesions that may lead to seizures. As seizures are persistent, I would recommend starting antiepileptic drugs. In this regard, considering your age, and the fact that you are overweight, I would suggest starting low dose Levetiracetam or Carbamazepine. You should discuss with your doctor on the above tests and treatment options. I hope you will find this answer helpful. I remain at your disposal for any further questions whenever you need. MRI brain, and EEG.')]

In [41]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
prompt = "I have started to get lots of acne on my face, particularly on my forehead what can I do"
# Define the prompt template
template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use three sentences maximum and keep the answer as concise as possible. 
Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
rag_prompt = PromptTemplate.from_template(template)
# Get the retrieved context
context = retriever.get_relevant_documents(prompt)
print("Retrieved context:")
for doc in context:
    print(doc)
# Construct the full prompt
full_prompt = rag_prompt.format(context=context, question=prompt)
print("Full prompt:", full_prompt)    


Retrieved context:
page_content='Hi there Acne has multifactorial etiology. Only acne soap does not improve if ypu have grade 2 or more grade acne. You need to have oral and topical medications. This before writing medicines i need to confirm your grade of acne. For mild grade topical clindamycin or retenoic acud derivative would suffice whereas for higher grade acne you need oral medicines aluke doxycycline azithromycin or isotretinoin. Acne vulgaris Cleansing face with antiacne face wash'
Full prompt: Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use three sentences maximum and keep the answer as concise as possible. 
Always say "thanks for asking!" at the end of the answer. 
[Document(page_content='Hi there Acne has multifactorial etiology. Only acne soap does not improve if ypu have grade 2 or more grade acne. You need to have oral and topical medications. This bef

In [42]:
# Generate the response using the appropriate method
response = watsonx_granite.generate([full_prompt])  # Use `generate` for multiple prompts


In [43]:
# Print the full response
print("Solution:", response)

Solution: generations=[[Generation(text='\nFor mild grade acne, you can use a topical clindamycin or retinoid derivative face wash. For higher grade acne, you will need oral medications such as aluke doxycycline, azithromycin, or isotretinoin. It is essential to consult a dermatologist to confirm the grade of your acne and recommend appropriate treatment.\n\nPlease note that acne has a multifactorial etiology, and only acne soap may not be sufficient for severe cases. Oral and topical medications are often required to manage acne effectively.\n\nThanks for asking!', generation_info={'finish_reason': 'eos_token'})]] llm_output={'token_usage': {'generated_token_count': 114, 'input_token_count': 214}, 'model_id': 'ibm/granite-13b-chat-v2'} run=[RunInfo(run_id=UUID('cd9beca3-1acc-4089-9f8b-ace76052cc5d'))]


## Ask your question
After preparing the documents, you can set up a chain to include them in a prompt. This will allow LLM to use the docs as a reference when preparing answers.
Get questions from the previously loaded dataset.

#### Adapting to Langchain Custom Retriev

In [44]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun

class CustomRetrieverLang(BaseRetriever): 
    def get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        # Perform the encoding and retrieval for a specific question
        ans = combined_pipe(query)
        ans = DataCollection(ans)
        answer=ans[0]['answer']
        answer_string = ' '.join(answer)
        return [Document(page_content=answer_string)]   
# Ensure correct VectorStoreRetriever usage
retriever = CustomRetrieverLang()

In [45]:
# Define the prompt template
template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use three sentences maximum and keep the answer as concise as possible. 
Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
rag_prompt = PromptTemplate.from_template(template)
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | watsonx_granite
)

In [46]:
prompt = "I have started to get lots of acne on my face, particularly on my forehead what can I do"
# Get the retrieved context
context = retriever.get_relevant_documents(prompt)
print("Retrieved context:")
for doc in context:
    print(doc)
# Construct the full prompt
full_prompt = rag_prompt.format(context=context, question=prompt)
print("Full prompt:", full_prompt)

print(rag_chain.invoke(prompt))  

### Simple prompt

In [54]:
prompt="I have acne. Can I use Pantoprazole with Cutein?"
print(rag_chain.invoke(prompt))

 No, you should not use Pantoprazole with Cutein. The document states that if you take both of these medications together, the iron in Cutein will not get absorbed. This can lead to increased acid in the stomach and indigestion. For acne treatment, it is recommended to use medications that do not interact with iron, such as isotretinoin or adapalene. Consult your dermatologist for more information.


## Detailed Prompts

In [55]:
prompt="I have acne. Can I use Pantoprazole with Cutein?"
# Get the retrieved context
context = retriever.get_relevant_documents(prompt)
print("Retrieved context:")
for doc in context:
    print(doc)
# Construct the full prompt
full_prompt = rag_prompt.format(context=context, question=prompt)
print("Full prompt:", full_prompt)  
# Generate the response using the appropriate method
response = watsonx_granite.generate([full_prompt])  # Use `generate` for multiple prompts
# Print the full response
print("Solution:", response)


Retrieved context:
page_content='Hi. Cutein tablet contains Iron as one of the ingredients, along with some other vitamins. The iron tablets can give increased acid in the stomach and indigestion. But, if you take both Cutein and Pantoprazole tablets together, the iron will not get absorbed. Therefore, first take Cutein tablet on empty stomach in the morning, and then 30 minutes later take Pantoprazole.'
Full prompt: Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use three sentences maximum and keep the answer as concise as possible. 
Always say "thanks for asking!" at the end of the answer. 
[Document(page_content='Hi. Cutein tablet contains Iron as one of the ingredients, along with some other vitamins. The iron tablets can give increased acid in the stomach and indigestion. But, if you take both Cutein and Pantoprazole tablets together, the iron will not get absorbed

## Release a Showcase

We've done an excellent job on the core functionality of our question answering engine. Now it's time to build a showcase with interface. [Gradio](https://gradio.app/) is a great tool for building demos. With Gradio, we simply need to wrap the data processing pipeline via a `chat` function:

In [56]:
import towhee
def chat(message, history):
    history = history or []
    response = rag_chain.invoke(message)
    history.append((message, response))
    return history, history

In [58]:
chat('I have acne. Can I use Pantoprazole with Cutein?',[])

([('I have acne. Can I use Pantoprazole with Cutein?',
   ' No, you should not use Pantoprazole with Cutein. The document states that if you take both of these medications together, the iron in Cutein will not get absorbed. This can lead to increased acid in the stomach and indigestion. For acne treatment, it is recommended to use medications that do not interact with iron, such as isotretinoin or adapalene. Consult your dermatologist for more information.')],
 [('I have acne. Can I use Pantoprazole with Cutein?',
   ' No, you should not use Pantoprazole with Cutein. The document states that if you take both of these medications together, the iron in Cutein will not get absorbed. This can lead to increased acid in the stomach and indigestion. For acne treatment, it is recommended to use medications that do not interact with iron, such as isotretinoin or adapalene. Consult your dermatologist for more information.')])

In [60]:
import gradio
collection.load()
chatbot = gradio.Chatbot()
interface = gradio.Interface(
    chat,
    ["text", "state"],
    [chatbot, "state"],
    allow_flagging="never",
)
interface.launch(inline=True, share=False)


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
